<a href="https://colab.research.google.com/github/bhanuchaddha/The-Ai-Handbook/blob/main/4-AI-Agents/customer_research_and_marketing_campaign_creator/Multi_Agent_Customer_Research_and_Campaign_Outreach_using_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install crewai crewai-tools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install crewai crewai-tools


In [4]:
from crewai import Agent, Task, Crew

In [17]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')
os.environ['SERPER_API_KEY']=userdata.get('SERPER_API_KEY')

In [18]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [19]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

In [20]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [21]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [22]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [23]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[search_tool],
    agent=lead_sales_rep_agent,
)

In [24]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=True,
	memory=True
)

In [25]:
inputs = {
    "lead_name": "Maersk",
    "industry": "Shipping",
    "key_decision_maker": "Vincent Clerc",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of Maersk, a company in the Shipping sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


# Agent: Sales Representative
## Thought: I need to gather detailed information about Maersk, including its background, key personnel, recent milestones, and identified needs. Since I can't access the internet directly for specific details about Maersk, I'll start by checking if there are any relevant instructions or documentation available in the directory.
## Using tool: List files in directory
## Tool Input: 
"{}"
## Tool Output: 
File paths: 
-./instructions/tech_startups_outreach.md
- ./instructions/e

In [26]:
from IPython.display import Markdown
Markdown(result.raw)

### Email Draft 1: Introduction to Sustainability Solutions

**Subject:** Partnering for a Sustainable Future

Dear Mr. Clerc,

I hope this message finds you well. Congratulations on Maersk's recent launch of the world's first carbon-neutral liner vessel and your new weekly ocean-rail offering! Your commitment to achieving net-zero emissions by 2050 is truly inspiring and positions Maersk as a leader in sustainable logistics.

At [Your Company Name], we are equally passionate about sustainable innovation. Our carbon tracking and reporting solutions can seamlessly integrate with your operations, providing critical insights to help you meet and exceed your sustainability goals. 

I would love the opportunity to discuss how we can support Maersk's journey towards greater environmental responsibility. Are you available for a brief call next week?

Looking forward to the opportunity to collaborate on this significant initiative.

Best regards,  
[Your Name]  
[Your Position]  
[Your Company Name]  
[Your Contact Information]  

---

### Email Draft 2: Focus on Digital Integration

**Subject:** Enhancing Efficiency Through Digital Transformation

Dear Mr. Clerc,

I wanted to reach out to commend Maersk on its recent strategic investments in digital tools, particularly with the launch of the new intermodal ship and rail transport solution. This proactive approach positions Maersk as a trailblazer in enhancing the efficiency of cargo transportation.

At [Your Company Name], we specialize in technology integration solutions that complement existing systems. Our platforms can support Maersk's digital transformation efforts, ensuring seamless operation and an enhanced customer experience.

I believe a collaboration could further empower your teams, especially in light of these recent initiatives. Would you be open to a conversation to explore this potential synergy?

Warm regards,  
[Your Name]  
[Your Position]  
[Your Company Name]  
[Your Contact Information]  

---

### Email Draft 3: Cost Efficiency and Optimization

**Subject:** Unlocking Cost Savings in Your Logistics Operations

Dear Mr. Clerc,

I hope you are doing well. It's exciting to witness Maersk's growth, particularly your recent achievements in sustainability and operational efficiency with the new weekly ocean-rail offering.

As you focus on continuous improvement, I wanted to introduce [Your Company Name]'s predictive analytics and automated systems. These solutions can identify optimization opportunities within your logistics operations, helping to drive down costs while maintaining the quality and standards your clients expect.

I would greatly appreciate the chance to discuss how we can work together to enhance the efficiency and effectiveness of Maersk's operations.

Best,  
[Your Name]  
[Your Position]  
[Your Company Name]  
[Your Contact Information]  

--- 

These tailored email drafts highlight Maersk's recent product launch and align our solutions with their sustainability and digital transformation goals.